In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [3]:
dic = {
    "Nazare-Surf-Report":194,
    "Pipeline-Backdoor-Surf-Report":616,
    "Mundaka-Surf-Report": 169,
    "El-Fronton-Surf-Report":1879,
    "The-Box-Surf-Report":2873,
    "Puerto-Escondido-Surf-Report":340,
    "Itacoatiara-Surf-Report":1273,
    "Padang-Padang-Surf-Report":1121,
    "Mosca-Point-Surf-Report":1875,
    "Tauro-Surf-Report":1878
       
}

In [24]:
nazare_ = ["Nazare"]
pipeline_ = ["Pipeline"]
mundaka_ = ["Mundaka"]
fronton_ = ["Fronton"]
box_ = ["The Box"]
puerto_escondido_ = ["Puerto Escondido"]
itacoatiara_ =  ["Itacoatiara"]
padang_padang_ =  ["Padang-Padang"]
kirra_ = ["Kirra"]
mosca_ = ["Mosca Point"]
tauro_ = ["Tauro"]


In [28]:
list_nazare = nazare_ * df_nazare_2019.shape[0]

list_pipeline = pipeline_ * df_nazare_2019.shape[0]

list_mundaka = mundaka_ * df_nazare_2019.shape[0]

list_fronton = fronton_ * df_nazare_2019.shape[0]

list_box = box_ * df_nazare_2019.shape[0]

list_puerto_escondido = puerto_escondido_ * df_nazare_2019.shape[0]

list_itacoatiara = itacoatiara_ * df_nazare_2019.shape[0]

list_padang_padang = padang_padang_ * df_nazare_2019.shape[0]

list_mosca = mosca_ * df_nazare_2019.shape[0]

list_tauro = tauro_ * df_nazare_2019.shape[0]

In [12]:
range_2019 = [("2019-01-01","2019-01-31"),("2019-02-01","2019-02-28"),("2019-03-01","2019-03-31"),("2019-04-01","2019-04-30"),("2019-05-01","2019-05-31"),("2019-06-01","2019-06-30"),("2019-07-01","2019-07-31"),("2019-08-01","2019-08-31"),("2019-09-01","2019-09-30"),("2019-10-01","2019-10-31"),("2019-11-01","2019-11-30"),("2019-12-01","2019-12-31")]

In [ ]:
ranges_2018 = [("2018-01-01","2018-01-31"),("2018-02-01","2018-02-28"),("2018-03-01","2018-03-31"),("2018-04-01","2018-04-30"),("2018-05-01","2018-05-31"),("2018-06-01","2018-06-30"),("2018-07-01","2018-07-31"),("2018-08-01","2018-08-31"),("2018-09-01","2018-09-30"),("2018-10-01","2018-10-31"),("2018-11-01","2018-11-30"),("2018-12-01","2018-12-31")]

In [73]:
def login():
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.get("https://magicseaweed.com/account/login/?dest=%2F")
    sleep(2)
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(2) > input").send_keys("lukesmith194@hotmail.com")
    driver.find_element_by_css_selector("#msw-js-user-login-tab > form > div:nth-child(3) > input").send_keys("Versus.123")
    sleep(1)
    driver.find_element_by_css_selector("#msw-js-login").click()
    sleep(2)

    return driver

In [74]:
driver = login()

In [75]:
def get_arrow_value(fila):
    col = fila.find_element_by_class_name("text-center.msw-js-tooltip.background-gray-lighter").get_attribute("data-original-title")
    direction, angle = col.split(" - ")
    
    angle = int(angle[:-1])
    
    return direction, angle

In [76]:
def get_wind_value(fila):
    try:
        col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-success").get_attribute("data-original-title")
    except:
        try:
            col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-warning").get_attribute("data-original-title")
        except:
            try:
                col = fila.find_element_by_class_name("text-center.last.msw-js-tooltip.td-square.background-danger").get_attribute("data-original-title")
            except:
                return "null", "null", np.nan
        
    
    text, angle = col.split(" - ")

    angle = int(angle[:-1])
    _, text = text.split(", ")

    text, direction = text.split()

    return text, direction, angle

In [77]:
def get_hora(fila):
    elems = fila.text.split("\n")
    elems = elems[0].split() + elems[1:]
    
    arrow = get_arrow_value(fila)
    wind = get_wind_value(fila)
    
    elems = elems + list(arrow) + list(wind)
    
    return elems

In [78]:
def get_dia(tabla):
    filas = tabla.find_elements_by_tag_name("tr")
    dia = filas[0].text.split()[1]
    
    # 6am 12pm 6pm son filas 4, 6, 8
    filas = filas[3:8:2]
    
    dias = [[dia] + get_hora(f) for f in filas]
    
    return dias

In [79]:
def get_range_dates(d, driver):
    # d is a tuple of 2 elements (start_date, end_date)
    # access webpage
        
    driver.get(f"https://magicseaweed.com/El-Fronton-Surf-Report/1879/Historic/?start={d[0]}&end={d[1]}")
    sleep(2)
    # extract 31 days' data
    table = driver.find_element_by_css_selector("#msw-js-fc > div.table-responsive-xs > table")

    tablitas = table.find_elements_by_tag_name("tbody")
    tablitas = tablitas[::3]

    mega_tabla = []

    for t in tablitas:
        dia = get_dia(t)
        mega_tabla.extend(dia)
        
    return mega_tabla

In [80]:
tabla_anio = []

for r in range_2019:
    print(f"Rango {r}...")
    driver.get(f"https://magicseaweed.com/El-Fronton-Surf-Report/1879/Historic/?start={r[0]}&end={r[1]}")
    
    tabla_mes = get_range_dates(r, driver)
    
    tabla_anio.extend(tabla_mes)

Rango ('2019-01-01', '2019-01-31')...
Rango ('2019-02-01', '2019-02-28')...
Rango ('2019-03-01', '2019-03-31')...
Rango ('2019-04-01', '2019-04-30')...
Rango ('2019-05-01', '2019-05-31')...
Rango ('2019-06-01', '2019-06-30')...
Rango ('2019-07-01', '2019-07-31')...
Rango ('2019-08-01', '2019-08-31')...
Rango ('2019-09-01', '2019-09-30')...
Rango ('2019-10-01', '2019-10-31')...
Rango ('2019-11-01', '2019-11-30')...
Rango ('2019-12-01', '2019-12-31')...


In [81]:
df_fronton_2019 = pd.DataFrame(tabla_anio, columns=[
    "date", "time", "range", "height", "period", "wind_mph", "wind_kmh", "mph", "temp", "prob",
    "swell_dir", "swell_angle", "wind", "wind_dir", "wind_angle",#"x1","x2","x3","x4"
])

ValueError: 15 columns passed, passed data had 19 columns

### Nazare

In [45]:
df_nazare_2019.head()

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle
0,01/01,6am,6-9ft,3.5ft,14s,3,4,mph,7°c,100%,W,267,Offshore,ESE,103
1,01/01,12pm,6-9ft,3ft,13s,1,2,mph,15°c,100%,W,270,Offshore,E,81
2,01/01,6pm,5-8ft,3ft,12s,4,5,mph,9°c,100%,W,275,Cross/Onshore,N,2
3,02/01,6am,4-7ft,2.5ft,11s,5,6,mph,7°c,100%,W,274,Cross/Onshore,N,10
4,02/01,12pm,3-5ft,2ft,10s,6,10,mph,14°c,100%,WSW,254,Onshore,NNW,343


In [71]:
pd.options.display.max_rows = 10000

In [72]:
df_mundaka_2019

,date,time,range,height,period,wind_mph,wind_kmh,mph,temp,prob,swell_dir,swell_angle,wind,wind_dir,wind_angle,x1,x2,x3,x4
0,01/01,6am,1ft,1ft,5s,0.8ft,9s,0.6ft,11s,3,3,mph,9°c,100%,N,7,Onshore,ENE,72.0
1,01/01,12pm,1ft,0.9ft,10s,1ft,4s,0.5ft,14s,4,3,mph,10°c,100%,NW,318,Onshore,NNE,14.0
2,01/01,6pm,1ft,1ft,4s,0.6ft,13s,0.6ft,10s,5,5,mph,9°c,100%,N,9,Onshore,NE,36.0
3,02/01,6am,1ft,0.7ft,13s,1ft,4s,0.8ft,12s,9,9,mph,8°c,100%,NW,312,Onshore,E,81.0
4,02/01,12pm,1ft,1.1ft,12s,1.3ft,4s,10,12,mph,9°c,100%,WNW,301,Onshore,E,83,None,NaN
5,02/01,6pm,1-2ft,1.7ft,4s,1.1ft,11s,0.4ft,16s,13,16,mph,10°c,100%,NNE,23,Onshore,ENE,76.0
6,03/01,6am,2ft,3ft,5s,1ft,10s,0.4ft,13s,13,14,mph,7°c,100%,NE,35,Onshore,E,82.0
7,03/01,12pm,2ft,2.5ft,5s,0.9ft,10s,0.4ft,12s,11,12,mph,7°c,100%,NE,35,Onshore,ENE,76.0
8,03/01,6pm,1-2ft,2.5ft,5s,0.8ft,9s,0.5ft,15s,12,14,mph,9°c,100%,NNE,33,Onshore,E,80.0
9,04/01,6am,2ft,2.5ft,6s,6,7,mph,6°c,100%,NNE,20,Onshore,E,81,None,None,None,NaN
